In [1]:
import xarray as xr
import pandas as pd
import time as tm
import os
import numpy as np
pd.set_option('display.float_format', '{:.6f}'.format)
np.set_printoptions(suppress=True)

In [2]:
inputpath = r"C:\Users\####3\Downloads\ERA5_reanalysis\tp2\2014\12" # keep single file only 
files = os.listdir(inputpath)
files

['dec.nc']

In [ ]:
dfs = {}
for file in files:
    print(f'{file}')
    ds = xr.open_mfdataset(f"{inputpath}\\{file}")
    rain = ds["tp"]       #write your precipitation variable
    time = rain["time"]
    
    date = []
    rain_v = []
    for i in range(time.shape[0]):
        rain_values = rain[i,:,:].values*1000         #this is done to convert data from m to mm
        rain_values = np.reshape(rain_values,(1,-1))
        rain_values = np.where(rain_values<=0 , 0 ,rain_values)
        rain_v.append(rain_values[0])
        date.append(time[i].values)
  
    data = pd.DataFrame(np.array(rain_v))
    data.index = date
    data.index = pd.to_datetime(data.index)
    daily_df = data.resample('D').sum()            #24-hourly data of day is added 
    dfs[file] = daily_df
    ds.close()  # Corrected to close the dataset
P_result_df = pd.concat(dfs.values())
P_result_df
P_result_df.index.name = "Date"
P_result_df.to_csv(r"C:\Users\####3\Downloads\ERA5_reanalysis\tp2\2014\csv\prec\P_2014_12.csv")
P_result_df

In [ ]:
#Location data 
lat = []
lon = []
for i in rain["latitude"]:
    for j in rain["longitude"]:
        lat.append(i.values)
        lon.append(j.values)

grid = list(range(0,len(lat)))
location_file = {"latitude":lat, "longitude":lon , "Grid": grid}
loc_data = pd.DataFrame(location_file)
loc_data.to_csv(r"C:\Users\####3\Downloads\ERA5_reanalysis\tp2\2014\csv\loc\loc.csv")
loc_data